# Test STXMAnalyzer with Real Data

This notebook tests the `STXMAnalyzer` in `src/echemistpy/processing/analyzers/stxm.py` using a real TXM HDF5 file.

In [ ]:
import sys
import os
from pathlib import Path
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

# Add src to path
project_root = Path("..").resolve()
if str(project_root / "src") not in sys.path:
    sys.path.append(str(project_root / "src"))

from echemistpy.io.loaders import load
from echemistpy.processing.analyzers.stxm import STXMAnalyzer

In [ ]:
# 1. Load Data
# Using the alpha MnO2 example file
file_path = project_root / "docs" / "examples" / "TXM" / "αMnO2" / "20230629_E1A_749.7x177.5y_specnorm_aliOF.hdf5.hdf5"
print(f"Loading: {file_path}")

raw_data, raw_info = load(file_path)

print("\nRaw Data Info:")
print(raw_info.to_dict())
print("\nRaw Data:")
print(raw_data.data)

In [ ]:
# 2. Analyze
analyzer = STXMAnalyzer(
    pca_components=6,
    perform_clustering=True,
    n_clusters=4,
    # clustering_method="kmeans",
    perform_fitting=True,
)

# Use analyze() method which handles validation, preprocessing, computation, and metadata inheritance
result_data, result_info = analyzer.analyze(raw_data, raw_info)

# 3. Inspect Results
print("\nAnalysis Info Parameters:")
print(result_info.parameters)

print("\nResult Dataset:")
ds_res = result_data.data
print(ds_res)


In [ ]:
# 4. Plot
ds_res = result_data.data

fig, ax = plt.subplots(1, 3, figsize=(15, 5))

# PCA Denoised Map (Sum)
if "denoised" in ds_res:
    if "energy" in ds_res.denoised.dims:
        img_sum = ds_res.denoised.sum(dim="energy")
        ax[0].imshow(img_sum, cmap="viridis")
        ax[0].set_title("Denoised Sum")
    else:
        ax[0].text(0.5, 0.5, "Denoised has no energy dim")
elif "optical_density" in ds_res:
    if "energy" in ds_res.optical_density.dims:
        img_sum = ds_res.optical_density.sum(dim="energy")
        ax[0].imshow(img_sum, cmap="viridis")
        ax[0].set_title("Original Sum")
else:
    ax[0].text(0.5, 0.5, "No Image Data")

# Cluster Labels
if "cluster_labels" in ds_res:
    ax[1].imshow(ds_res.cluster_labels, cmap="tab10")
    ax[1].set_title("Cluster Labels")
else:
    ax[1].text(0.5, 0.5, "No Cluster Labels")

# Cluster Centroids / Spectra
if hasattr(result_info, "others") and "cluster_centroids" in result_info.others:
    centroids = result_info.others["cluster_centroids"]
    if "energy" in ds_res.coords:
        energy = ds_res.energy
        for i, cent in enumerate(centroids):
            ax[2].plot(energy, cent, label=f"Cluster {i}")

        ax[2].legend()
        ax[2].set_title("Cluster Spectra")
    else:
        ax[2].text(0.5, 0.5, "No energy axis")
else:
    ax[2].text(0.5, 0.5, "No centroids")

plt.tight_layout()
plt.show()